In [1]:
!pip install pyngrok streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 1.1 MB/s eta 0:00:00


In [2]:
code = '''
import streamlit as st
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import ast


encoders = pd.read_csv("/content/drive/MyDrive/breast_cancer_detection/encoders.csv")
encoders_dict = {}
for column in encoders.columns:
    encoders_dict[column] = dict(encoders[column].dropna().to_dict())

image_view = []
left_or_right_breast=[]
calc_type=[]
calc_distribution=[]
for feature, data in encoders_dict.items():
    if 1 in data:
        feature_dict_str = data[1].replace('nan', 'None')
        feature_dict = eval(feature_dict_str)
        feature_list = list(feature_dict.keys())
        globals()[feature] = feature_list

special_labels = ['assessment', 'breast_density']

binary_model = load_model("/content/drive/MyDrive/breast_cancer_detection/binary_classification_model.h5")
multi_label_model = load_model("/content/drive/MyDrive/breast_cancer_detection/multi_label_classification_model.h5")

def preprocess_image(uploaded_image):
    img = cv2.imdecode(np.frombuffer(uploaded_image.read(), np.uint8), -1)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Convert to 3 channels (RGB)
    img = cv2.resize(img, (224, 224))  # Resize to the correct shape
    img = img / 255.0  # Normalize to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension (1, 224, 224, 3)
    return img

def predict_malignant_or_benign(image):
    binary_prediction = binary_model.predict(image)
    if binary_prediction > 0.45:
        return "Malignant"
    else:
        return "Benign"

def predict_other_features(image):
    # Predict using the multi-label model
    multi_label_prediction = multi_label_model.predict(image)

    predictions = []
    for idx, prediction in enumerate(multi_label_prediction):
        if idx == 5:
            predictions.append("assessment number: " + str(np.argmax(prediction)))
        elif idx == 4:
            predictions.append("breast_density number: " + str(np.argmax(prediction)))
        else:
            max_index = np.argmax(prediction)
            if idx == 0:
                predictions.append(f"image_view: {image_view[max_index]}")
            elif idx == 1:

                predictions.append(f"left_or_right_breast: {left_or_right_breast[max_index]}")
            elif idx == 2:
                predictions.append(f"calc_type: {calc_type[max_index]}")
            elif idx == 3:
                if predictions[2].split(':')[1]== ' None':
                   predictions.append(f"calc_distribution: None")
                else:
                   predictions.append(f"calc_distribution: {calc_distribution[max_index]}")

    return predictions


st.title("Breast Cancer Detection")
st.write("Upload a breast tumor image for prediction")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png"])

if uploaded_file is not None:
    image = preprocess_image(uploaded_file)

    st.subheader("Malignant or Benign Prediction:")
    result = predict_malignant_or_benign(image)
    st.write(f"Prediction: {result}")

    if result == "Malignant":
        st.subheader("Other Features Prediction (Multi-label):")
        features = predict_other_features(image)

        for feature in features:
            st.write(f"- {feature}")

        st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)

    st.markdown("""
        <style>
            .css-1v0mbdj {background-color: #f7f7f7; padding: 10px; border-radius: 10px;}
            h1 {color: #6a1b9a;}
            h2 {color: #d32f2f;}
            .stButton button {background-color: #6a1b9a; color: white; border-radius: 5px;}
        </style>
    """, unsafe_allow_html=True)
'''

with open("app.py", "w") as file:
    file.write(code)


In [7]:
from pyngrok import ngrok
import os
ngrok.set_auth_token("2po83xsmq48ye6YRys2U9EeM6Ke_87y32DWJ2exfm264W7u37")


public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at: {public_url}")
!streamlit run app.py &


Streamlit app is live at: NgrokTunnel: "https://f08a-35-196-96-240.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.96.240:8501

2025-01-19 23:59:34.427625: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-19 23:59:34.473448: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-19 23:59:34.488928: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 23:59:36.583431: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [85]:
import streamlit as st
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import ast


encoders = pd.read_csv("/content/drive/MyDrive/breast_cancer_detection/encoders.csv")
encoders_dict = {}
for column in encoders.columns:
    encoders_dict[column] = dict(encoders[column].dropna().to_dict())

image_view = []
left_or_right_breast=[]
calc_type=[]
calc_distribution=[]
for feature, data in encoders_dict.items():
    if 1 in data:
        feature_dict_str = data[1].replace('nan', 'None')
        feature_dict = eval(feature_dict_str)
        feature_list = list(feature_dict.keys())
        globals()[feature] = feature_list

special_labels = ['assessment', 'breast_density']

binary_model = load_model("/content/drive/MyDrive/breast_cancer_detection/binary_classification_model.h5")
multi_label_model = load_model("/content/drive/MyDrive/breast_cancer_detection/multi_label_classification_model.h5")

def preprocess_image(uploaded_image):
    img = cv2.imdecode(np.frombuffer(uploaded_image.read(), np.uint8), -1)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Convert to 3 channels (RGB)
    img = cv2.resize(img, (224, 224))  # Resize to the correct shape
    img = img / 255.0  # Normalize to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension (1, 224, 224, 3)
    return img

def predict_malignant_or_benign(image):
    binary_prediction = binary_model.predict(image)
    if binary_prediction <= 0.5:
        return "Malignant"
    else:
        return "Benign"

def predict_other_features(image):
    # Predict using the multi-label model
    multi_label_prediction = multi_label_model.predict(image)

    predictions = []
    for idx, prediction in enumerate(multi_label_prediction):
        if idx == 5:
            predictions.append("assessment number: " + str(np.argmax(prediction)))
        elif idx == 4:
            predictions.append("breast_density number: " + str(np.argmax(prediction)))
        else:
            max_index = np.argmax(prediction)
            if idx == 0:
                predictions.append(f"image_view: {image_view[max_index]}")
            elif idx == 1:

                predictions.append(f"left_or_right_breast: {left_or_right_breast[max_index]}")
            elif idx == 2:
                predictions.append(f"calc_type: {calc_type[max_index]}")
            elif idx == 3:
                if predictions[2].split(':')[1]== None:
                   predictions.append(f"calc_distribution: None")
                else:
                   predictions.append(f"calc_distribution: {calc_distribution[max_index]}")

    return predictions

In [132]:
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# Open and preprocess the image
with open(image_path, "rb") as f:
    processed_image = preprocess_image(f)

# Run predictions
malignant_or_benign = predict_malignant_or_benign(processed_image)
multi_label_predictions = predict_other_features(processed_image)

Saving C_0365_1.RIGHT_CC.jpg to C_0365_1.RIGHT_CC.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step


In [114]:
multi_label_predictions

['image_view: MLO',
 'left_or_right_breast: LEFT',
 'calc_type: VASCULAR',
 'calc_distribution: None',
 'breast_density number: 1',
 'assessment number: 2']

In [133]:
malignant_or_benign

'Malignant'